<a href="https://colab.research.google.com/github/nilesh3030/Natural-Language-Processing/blob/main/nlpdemystified_transformers_and_pretraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Natural Language Processing Demystified | Transformers, Pre-training, and Transfer Learning
https://nlpdemystified.org<br>
https://github.com/nitinpunjabi/nlp-demystified<br><br>

Course module for this demo: https://www.nlpdemystified.org/course/transformers

**IMPORTANT**<br>
Enable **GPU acceleration** by going to *Runtime > Change Runtime Type*. Keep in mind that, on certain tiers, you're not guaranteed GPU access depending on usage history and current load.
<br><br>
Also, if you're running this in the cloud rather than a local Jupyter server on your machine, then the notebook will *timeout* after a period of inactivity.
<br><br>
Refer to this link on how to run Colab notebooks locally on your machine to avoid this issue:<br>
https://research.google.com/colaboratory/local-runtimes.html

In [1]:
!pip install BPEmb

import math
import numpy as np
import tensorflow as tf

from bpemb import BPEmb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.7 MB/s eta 0:00:00


# Transformers From Scratch

We'll build a transformer from scratch, layer-by-layer. We'll start with the **Multi-Head Self-Attention** layer since that's the most involved bit. Once we have that working, the rest of the model will look familiar if you've been following the course so far.

## Multi-Head Self-Attention

#### Scaled Dot Product Self-Attention


Inside each attention head is a **Scaled Dot Product Self-Attention** operation as we covered in the slides. Given *queries*, *keys*, and *values*, the operation returns a new "mix" of the values.

$$Attention(Q, K, V) = softmax(\frac{QK^T)}{\sqrt{d_k}})V$$

The following function implements this and also takes a mask to account for padding and for masking future tokens for decoding (i.e. **look-ahead mask**). We'll cover masking later in the notebook.

In [2]:
def scaled_dot_product_attention(query, key, value, mask=None):
  key_dim = tf.cast(tf.shape(key)[-1], tf.float32)
  scaled_scores = tf.matmul(query, key, transpose_b=True) / np.sqrt(key_dim)

  if mask is not None:
    scaled_scores = tf.where(mask==0, -np.inf, scaled_scores)

  softmax = tf.keras.layers.Softmax()
  weights = softmax(scaled_scores) 
  return tf.matmul(weights, value), weights

Suppose our *queries*, *keys*, and *values* are each a length of 3 with a dimension of 4.

In [3]:
seq_len = 3
embed_dim = 4

queries = np.random.rand(seq_len, embed_dim)
keys = np.random.rand(seq_len, embed_dim)
values = np.random.rand(seq_len, embed_dim)

print("Queries:\n", queries)

Queries:
 [[0.6237169  0.34023532 0.61491834 0.2828286 ]
 [0.91863068 0.92249488 0.48096484 0.2290104 ]
 [0.22862767 0.69082455 0.58926884 0.40563778]]


This would be the self-attention output and weights.

In [4]:
output, attn_weights = scaled_dot_product_attention(queries, keys, values)

print("Output\n", output, "\n")
print("Weights\n", attn_weights)

Output
 tf.Tensor(
[[0.540856   0.60524607 0.6178887  0.43132782]
 [0.56730306 0.61340505 0.6434348  0.43183142]
 [0.51982206 0.6192749  0.6330088  0.42603007]], shape=(3, 4), dtype=float32) 

Weights
 tf.Tensor(
[[0.32830736 0.30138662 0.37030604]
 [0.3861844  0.2964321  0.31738344]
 [0.32258734 0.33413061 0.34328204]], shape=(3, 3), dtype=float32)


#### Generating queries, keys, and values for multiple heads.

Now that we have a way to calculate self-attention, let's actually generate the input *queries*, *keys*, and *values* for multiple heads.
<br><br>
In the slides (and in most references), each attention head had its <u>own separate</u> set of *query*, *key*, and *value* weights. Each weight matrix was of dimension $d\ x \ d/h$ where h was the number of heads. 

![](https://drive.google.com/uc?export=view&id=1SLWkHQgy4nQPFvvjG5_V8UTtpSAJ2zrr)

It's easier to understand things this way and we can certainly code it this way as well. But we can also "simulate" different heads with a single query matrix, single key matrix, and single value matrix.
<br><br>
We'll do both. First we'll create *query*, *key*, and *value* vectors using separate weights per head.
<br><br>
In the slides, we used an example of 12 dimensional embeddings processed by  three attentions heads, and we'll do the same here.

In [5]:
batch_size = 1
seq_len = 3
embed_dim = 12
num_heads = 3
head_dim = embed_dim // num_heads

print(f"Dimension of each head: {head_dim}")

Dimension of each head: 4


**Using separate weight matrices per head**

Suppose these are our input embeddings. Here we have a batch of 1 containing a sequence of length 3, with each element being a 12-dimensional embedding.

In [6]:
x = np.random.rand(batch_size, seq_len, embed_dim).round(1)
print("Input shape: ", x.shape, "\n")
print("Input:\n", x)

Input shape:  (1, 3, 12) 

Input:
 [[[0.6 0.4 0.9 0.6 0.1 1.  1.  0.4 0.1 0.  0.9 0.4]
  [0.8 0.2 0.6 0.4 0.9 0.2 0.6 0.8 0.3 0.7 0.2 0.1]
  [0.1 0.9 0.1 0.  0.9 0.8 0.4 0.8 0.1 0.4 0.9 0.6]]]


We'll declare three sets of *query* weights (one for each head), three sets of *key* weights, and three sets of *value* weights. Remember each weight matrix should have a dimension of $\text{d}\ \text{x}\ \text{d/h}$.

In [7]:
# The query weights for each head.
wq0 = np.random.rand(embed_dim, head_dim).round(1)
wq1 = np.random.rand(embed_dim, head_dim).round(1)
wq2 = np.random.rand(embed_dim, head_dim).round(1)

# The key weights for each head. 
wk0 = np.random.rand(embed_dim, head_dim).round(1)
wk1 = np.random.rand(embed_dim, head_dim).round(1)
wk2 = np.random.rand(embed_dim, head_dim).round(1)

# The value weights for each head.
wv0 = np.random.rand(embed_dim, head_dim).round(1)
wv1 = np.random.rand(embed_dim, head_dim).round(1)
wv2 = np.random.rand(embed_dim, head_dim).round(1)

In [8]:
print("The three sets of query weights (one for each head):")
print("wq0:\n", wq0)
print("wq1:\n", wq1)
print("wq2:\n", wq1)

The three sets of query weights (one for each head):
wq0:
 [[0.6 0.6 0.4 0.6]
 [0.9 0.2 0.1 0.7]
 [0.5 0.3 0.6 0.7]
 [0.3 0.3 0.7 0.4]
 [0.5 0.  1.  0.1]
 [0.4 0.7 0.1 0.6]
 [0.2 0.9 0.9 0.7]
 [0.5 0.2 0.8 0.4]
 [0.3 0.3 0.3 0.3]
 [0.  0.2 0.5 0.6]
 [0.8 0.9 0.2 0.7]
 [0.7 0.4 0.4 0.8]]
wq1:
 [[0.7 0.9 0.6 0.4]
 [0.4 0.7 0.5 0.5]
 [0.2 0.5 0.9 0. ]
 [0.1 0.4 0.8 0.5]
 [0.1 0.3 0.8 0. ]
 [0.4 0.6 0.3 0.4]
 [0.  0.2 0.4 0. ]
 [0.8 0.6 0.  0.9]
 [0.7 0.4 0.4 0.4]
 [0.6 0.9 0.7 0.2]
 [0.1 0.2 0.7 0.3]
 [0.7 0.5 0.8 0.7]]
wq2:
 [[0.7 0.9 0.6 0.4]
 [0.4 0.7 0.5 0.5]
 [0.2 0.5 0.9 0. ]
 [0.1 0.4 0.8 0.5]
 [0.1 0.3 0.8 0. ]
 [0.4 0.6 0.3 0.4]
 [0.  0.2 0.4 0. ]
 [0.8 0.6 0.  0.9]
 [0.7 0.4 0.4 0.4]
 [0.6 0.9 0.7 0.2]
 [0.1 0.2 0.7 0.3]
 [0.7 0.5 0.8 0.7]]


We'll generate our *queries*, *keys*, and *values* for each head by multiplying our input by the weights.

In [10]:
# Geneated queries, keys, and values for the first head.
q0 = np.dot(x, wq0)
k0 = np.dot(x, wk0)
v0 = np.dot(x, wv0)

# Geneated queries, keys, and values for the second head.
q1 = np.dot(x, wq1)
k1 = np.dot(x, wk1)
v1 = np.dot(x, wv1)

# Geneated queries, keys, and values for the third head.
q2 = np.dot(x, wq2)
k2 = np.dot(x, wk2)
v2 = np.dot(x, wv2)

These are the resulting *query*, *key*, and *value* vectors for the first head.

In [11]:
print("Q, K, and V for first head:\n")

print(f"q0 {q0.shape}:\n", q0, "\n")
print(f"k0 {k0.shape}:\n", k0, "\n")
print(f"v0 {v0.shape}:\n", v0)

Q, K, and V for first head:

q0 (1, 3, 4):
 [[[3.23 3.57 3.03 3.96]
  [2.45 2.11 3.6  2.88]
  [3.34 2.51 2.82 3.31]]] 

k0 (1, 3, 4):
 [[[4.21 3.26 3.05 2.74]
  [2.92 2.63 2.71 2.99]
  [3.48 2.74 2.58 2.66]]] 

v0 (1, 3, 4):
 [[[3.16 3.03 2.35 5.08]
  [2.49 2.8  2.34 4.83]
  [2.94 3.46 2.99 4.89]]]


Now that we have our Q, K, V vectors, we can just pass them to our self-attention operation. Here we're calculating the output and attention weights for the first head.

In [12]:
out0, attn_weights0 = scaled_dot_product_attention(q0, k0, v0)

print("Output from first attention head: ", out0, "\n")
print("Attention weights from first head: ", attn_weights0)

Output from first attention head:  tf.Tensor(
[[[3.1253777 3.0417163 2.3795164 5.062043 ]
  [3.0960062 3.0469806 2.3986073 5.0478077]
  [3.116835  3.0465755 2.3893042 5.0571923]]], shape=(1, 3, 4), dtype=float32) 

Attention weights from first head:  tf.Tensor(
[[[0.9169673  0.03634553 0.04668728]
  [0.85273945 0.07021441 0.07704613]
  [0.89386564 0.04403375 0.06210062]]], shape=(1, 3, 3), dtype=float32)


Here are the other two (attention weights are ignored).

In [13]:
out1, _ = scaled_dot_product_attention(q1, k1, v1)
out2, _ = scaled_dot_product_attention(q2, k2, v2)

print("Output from second attention head: ", out1, "\n")
print("Output from third attention head: ", out2,)

Output from second attention head:  tf.Tensor(
[[[2.768069  3.8823671 2.5511048 2.6974177]
  [2.7745962 3.8707633 2.513121  2.666007 ]
  [2.767774  3.8762562 2.533748  2.6857417]]], shape=(1, 3, 4), dtype=float32) 

Output from third attention head:  tf.Tensor(
[[[2.0942488 2.775998  2.155232  2.8350935]
  [2.0935578 2.7601702 2.1386647 2.85378  ]
  [2.0937302 2.7649665 2.1433175 2.84821  ]]], shape=(1, 3, 4), dtype=float32)


As we covered in the slides, once we have each head's output, we concatenate them and then put them through a linear layer for further processing.

In [14]:
combined_out_a = np.concatenate((out0, out1, out2), axis=-1)
print(f"Combined output from all heads {combined_out_a.shape}:")
print(combined_out_a)

# The final step would be to run combined_out_a through a linear/dense layer 
# for further processing.

Combined output from all heads (1, 3, 12):
[[[3.1253777 3.0417163 2.3795164 5.062043  2.768069  3.8823671 2.5511048
   2.6974177 2.0942488 2.775998  2.155232  2.8350935]
  [3.0960062 3.0469806 2.3986073 5.0478077 2.7745962 3.8707633 2.513121
   2.666007  2.0935578 2.7601702 2.1386647 2.85378  ]
  [3.116835  3.0465755 2.3893042 5.0571923 2.767774  3.8762562 2.533748
   2.6857417 2.0937302 2.7649665 2.1433175 2.84821  ]]]


So that's a complete run of **multi-head self-attention** using separate sets of weights per head.<br>

Let's now get the same thing done using a single query weight matrix, single key weight matrix, and single value weight matrix.<br><br>
These were our separate per-head query weights:

In [15]:
print("Query weights for first head: \n", wq0, "\n")
print("Query weights for second head: \n", wq1, "\n")
print("Query weights for third head: \n", wq2)

Query weights for first head: 
 [[0.6 0.6 0.4 0.6]
 [0.9 0.2 0.1 0.7]
 [0.5 0.3 0.6 0.7]
 [0.3 0.3 0.7 0.4]
 [0.5 0.  1.  0.1]
 [0.4 0.7 0.1 0.6]
 [0.2 0.9 0.9 0.7]
 [0.5 0.2 0.8 0.4]
 [0.3 0.3 0.3 0.3]
 [0.  0.2 0.5 0.6]
 [0.8 0.9 0.2 0.7]
 [0.7 0.4 0.4 0.8]] 

Query weights for second head: 
 [[0.7 0.9 0.6 0.4]
 [0.4 0.7 0.5 0.5]
 [0.2 0.5 0.9 0. ]
 [0.1 0.4 0.8 0.5]
 [0.1 0.3 0.8 0. ]
 [0.4 0.6 0.3 0.4]
 [0.  0.2 0.4 0. ]
 [0.8 0.6 0.  0.9]
 [0.7 0.4 0.4 0.4]
 [0.6 0.9 0.7 0.2]
 [0.1 0.2 0.7 0.3]
 [0.7 0.5 0.8 0.7]] 

Query weights for third head: 
 [[0.5 0.7 0.4 0.7]
 [0.5 0.7 0.4 0.1]
 [0.8 0.  0.4 0.3]
 [0.4 0.  0.6 0.7]
 [0.6 0.6 0.5 0.2]
 [0.2 0.4 0.7 0.6]
 [0.7 0.8 0.1 0.2]
 [0.5 1.  0.  0.7]
 [0.1 0.7 0.7 0.9]
 [0.7 0.1 0.  0.8]
 [0.3 0.7 0.2 0.2]
 [0.7 0.3 0.4 0.1]]


Suppose instead of declaring three separate query weight matrices, we had declared one. i.e. a single $d\ x\ d$ matrix. We're concatenating our per-head query weights here instead of declaring a new set of weights so that we get the same results.

In [16]:
wq = np.concatenate((wq0, wq1, wq2), axis=1)
print(f"Single query weight matrix {wq.shape}: \n", wq)

Single query weight matrix (12, 12): 
 [[0.6 0.6 0.4 0.6 0.7 0.9 0.6 0.4 0.5 0.7 0.4 0.7]
 [0.9 0.2 0.1 0.7 0.4 0.7 0.5 0.5 0.5 0.7 0.4 0.1]
 [0.5 0.3 0.6 0.7 0.2 0.5 0.9 0.  0.8 0.  0.4 0.3]
 [0.3 0.3 0.7 0.4 0.1 0.4 0.8 0.5 0.4 0.  0.6 0.7]
 [0.5 0.  1.  0.1 0.1 0.3 0.8 0.  0.6 0.6 0.5 0.2]
 [0.4 0.7 0.1 0.6 0.4 0.6 0.3 0.4 0.2 0.4 0.7 0.6]
 [0.2 0.9 0.9 0.7 0.  0.2 0.4 0.  0.7 0.8 0.1 0.2]
 [0.5 0.2 0.8 0.4 0.8 0.6 0.  0.9 0.5 1.  0.  0.7]
 [0.3 0.3 0.3 0.3 0.7 0.4 0.4 0.4 0.1 0.7 0.7 0.9]
 [0.  0.2 0.5 0.6 0.6 0.9 0.7 0.2 0.7 0.1 0.  0.8]
 [0.8 0.9 0.2 0.7 0.1 0.2 0.7 0.3 0.3 0.7 0.2 0.2]
 [0.7 0.4 0.4 0.8 0.7 0.5 0.8 0.7 0.7 0.3 0.4 0.1]]


In the same vein, pretend we declared a single key weight matrix, and single value weight matrix.

In [17]:
wk = np.concatenate((wk0, wk1, wk2), axis=1)
wv = np.concatenate((wv0, wv1, wv2), axis=1)

print(f"Single key weight matrix {wk.shape}:\n", wk, "\n")
print(f"Single value weight matrix {wv.shape}:\n", wv)

Single key weight matrix (12, 12):
 [[0.8 0.7 0.8 0.6 0.4 0.4 0.  0.8 0.6 0.8 0.8 0.8]
 [0.5 0.9 0.5 0.4 0.4 0.  0.5 0.3 0.5 0.5 0.  0.8]
 [0.6 0.8 0.2 1.  0.2 0.6 0.1 0.6 0.3 0.4 0.3 0.9]
 [0.3 0.6 0.7 0.1 0.2 0.2 0.8 0.4 0.8 0.7 0.1 0.2]
 [0.3 0.5 0.1 0.7 0.9 0.  0.  0.3 0.5 0.7 0.2 0.3]
 [1.  0.5 0.3 0.8 0.4 0.5 0.5 1.  0.3 0.  0.8 0.3]
 [0.7 0.3 0.5 0.  0.2 0.4 0.3 0.2 0.2 0.7 0.7 0.1]
 [0.2 0.1 0.4 0.2 0.9 0.4 0.8 0.1 0.  0.2 0.2 0.6]
 [0.  0.6 0.7 0.5 0.6 0.3 0.  0.6 0.5 0.8 0.2 0.7]
 [0.6 0.1 0.6 0.9 0.5 0.9 0.2 0.7 1.  0.4 0.4 0.5]
 [0.8 0.1 0.5 0.2 0.9 1.  0.4 0.1 0.8 0.7 0.5 0.6]
 [0.7 0.9 0.7 0.2 0.6 0.9 0.7 0.3 0.8 1.  0.1 0.2]] 

Single value weight matrix (12, 12):
 [[0.8 0.4 0.1 0.8 0.2 0.8 0.4 1.  0.3 0.8 0.2 0.2]
 [0.6 0.9 0.3 0.7 0.9 0.7 0.5 0.3 0.1 0.2 0.9 0.4]
 [0.4 0.1 0.5 0.9 0.2 0.7 0.5 0.3 0.3 0.5 0.7 0.6]
 [0.6 0.9 0.2 0.7 0.8 1.  0.8 0.3 0.2 0.8 0.5 0.1]
 [0.6 0.9 0.7 1.  0.3 0.6 0.3 0.2 0.1 0.5 0.2 0.9]
 [0.5 0.1 0.1 0.3 0.6 0.2 0.1 0.5 0.8 0.3 0.3 0.6]
 [0.1

Now we can calculate all our *queries*, *keys*, and *values* with three dot products.

In [18]:
q_s = np.dot(x, wq)
k_s = np.dot(x, wk)
v_s = np.dot(x, wv)

These are our resulting query vectors (we'll call them "combined queries"). How do we simulate different heads with this?

In [19]:
print(f"Query vectors using a single weight matrix {q_s.shape}:\n", q_s)

Query vectors using a single weight matrix (1, 3, 12):
 [[[3.23 3.57 3.03 3.96 1.99 3.   3.62 2.09 3.18 3.18 2.38 2.56]
  [2.45 2.11 3.6  2.88 2.33 3.15 3.29 1.81 3.19 3.05 1.82 2.9 ]
  [3.34 2.51 2.82 3.31 2.32 2.96 3.15 2.34 2.94 3.6  1.98 2.14]]]


Somehow, we need to separate these vectors such they're treated like three separate sets by the self-attention operation.

In [20]:
print(q0, "\n")
print(q1, "\n")
print(q2)

[[[3.23 3.57 3.03 3.96]
  [2.45 2.11 3.6  2.88]
  [3.34 2.51 2.82 3.31]]] 

[[[1.99 3.   3.62 2.09]
  [2.33 3.15 3.29 1.81]
  [2.32 2.96 3.15 2.34]]] 

[[[3.18 3.18 2.38 2.56]
  [3.19 3.05 1.82 2.9 ]
  [2.94 3.6  1.98 2.14]]]


Notice how each set of per-head queries looks like we took the combined queries, and chopped them vertically every four dimensions.
<br><br>
We can split our combined queries into $\text{d}\ \text{x}\ \text{d/h}$ heads using **reshape** and **transpose**.<br><br>
The first step is to *reshape* our combined queries from a shape of:<br>
(batch_size, seq_len, embed_dim)<br>

into a shape of<br>
 (batch_size, seq_len, num_heads, head_dim).
 <br>

 https://www.tensorflow.org/api_docs/python/tf/reshape

In [21]:
# Note: we can achieve the same thing by passing -1 instead of seq_len.
q_s_reshaped = tf.reshape(q_s, (batch_size, seq_len, num_heads, head_dim))
print(f"Combined queries: {q_s.shape}\n", q_s, "\n")
print(f"Reshaped into separate heads: {q_s_reshaped.shape}\n", q_s_reshaped)

Combined queries: (1, 3, 12)
 [[[3.23 3.57 3.03 3.96 1.99 3.   3.62 2.09 3.18 3.18 2.38 2.56]
  [2.45 2.11 3.6  2.88 2.33 3.15 3.29 1.81 3.19 3.05 1.82 2.9 ]
  [3.34 2.51 2.82 3.31 2.32 2.96 3.15 2.34 2.94 3.6  1.98 2.14]]] 

Reshaped into separate heads: (1, 3, 3, 4)
 tf.Tensor(
[[[[3.23 3.57 3.03 3.96]
   [1.99 3.   3.62 2.09]
   [3.18 3.18 2.38 2.56]]

  [[2.45 2.11 3.6  2.88]
   [2.33 3.15 3.29 1.81]
   [3.19 3.05 1.82 2.9 ]]

  [[3.34 2.51 2.82 3.31]
   [2.32 2.96 3.15 2.34]
   [2.94 3.6  1.98 2.14]]]], shape=(1, 3, 3, 4), dtype=float64)


At this point, we have our desired shape. The next step is to *transpose* it such that simulates vertically chopping our combined queries. By transposing, our matrix dimensions become:<br>
(batch_size, num_heads, seq_len, head_dim)<br>

https://www.tensorflow.org/api_docs/python/tf/transpose

In [22]:
q_s_transposed = tf.transpose(q_s_reshaped, perm=[0, 2, 1, 3]).numpy()
print(f"Queries transposed into \"separate\" heads {q_s_transposed.shape}:\n", 
      q_s_transposed)

Queries transposed into "separate" heads (1, 3, 3, 4):
 [[[[3.23 3.57 3.03 3.96]
   [2.45 2.11 3.6  2.88]
   [3.34 2.51 2.82 3.31]]

  [[1.99 3.   3.62 2.09]
   [2.33 3.15 3.29 1.81]
   [2.32 2.96 3.15 2.34]]

  [[3.18 3.18 2.38 2.56]
   [3.19 3.05 1.82 2.9 ]
   [2.94 3.6  1.98 2.14]]]]


If we compare this against the separate per-head queries we calculated previously, we see the same result except we now have all our queries in a single matrix.

In [23]:
print("The separate per-head query matrices from before: ")
print(q0, "\n")
print(q1, "\n")
print(q2)

The separate per-head query matrices from before: 
[[[3.23 3.57 3.03 3.96]
  [2.45 2.11 3.6  2.88]
  [3.34 2.51 2.82 3.31]]] 

[[[1.99 3.   3.62 2.09]
  [2.33 3.15 3.29 1.81]
  [2.32 2.96 3.15 2.34]]] 

[[[3.18 3.18 2.38 2.56]
  [3.19 3.05 1.82 2.9 ]
  [2.94 3.6  1.98 2.14]]]


Let's do the exact same thing with our combined keys and values.

In [24]:
k_s_transposed = tf.transpose(tf.reshape(k_s, (batch_size, -1, num_heads, head_dim)), perm=[0, 2, 1, 3]).numpy()
v_s_transposed = tf.transpose(tf.reshape(v_s, (batch_size, -1, num_heads, head_dim)), perm=[0, 2, 1, 3]).numpy()

print(f"Keys for all heads in a single matrix {k_s.shape}: \n", k_s_transposed, "\n")
print(f"Values for all heads in a single matrix {v_s.shape}: \n", v_s_transposed)

Keys for all heads in a single matrix (1, 3, 12): 
 [[[[4.21 3.26 3.05 2.74]
   [2.92 2.63 2.71 2.99]
   [3.48 2.74 2.58 2.66]]

  [[2.86 3.25 2.53 2.92]
   [3.1  2.43 1.69 2.61]
   [3.78 2.81 2.48 2.25]]

  [[2.95 3.42 2.92 3.09]
   [2.8  3.23 2.23 2.99]
   [2.96 3.11 2.06 2.85]]]] 

Values for all heads in a single matrix (1, 3, 12): 
 [[[[3.16 3.03 2.35 5.08]
   [2.49 2.8  2.34 4.83]
   [2.94 3.46 2.99 4.89]]

  [[2.73 3.97 2.83 2.92]
   [2.3  3.61 2.1  2.77]
   [2.88 3.76 2.12 2.31]]

  [[2.1  2.87 2.27 2.72]
   [2.05 2.8  1.73 2.92]
   [2.12 2.17 2.04 3.42]]]]


Set up this way, we can now calculate the outputs from all attention heads with a single call to our self-attention operation.

In [25]:
all_heads_output, all_attn_weights = scaled_dot_product_attention(q_s_transposed, 
                                                                  k_s_transposed, 
                                                                  v_s_transposed)
print("Self attention output:\n", all_heads_output)

Self attention output:
 tf.Tensor(
[[[[3.1253777 3.0417163 2.3795164 5.062043 ]
   [3.0960062 3.0469806 2.3986073 5.0478077]
   [3.116835  3.0465755 2.3893042 5.0571923]]

  [[2.768069  3.8823671 2.5511048 2.6974177]
   [2.7745962 3.8707633 2.513121  2.666007 ]
   [2.767774  3.8762562 2.533748  2.6857417]]

  [[2.0942488 2.775998  2.155232  2.8350935]
   [2.0935578 2.7601702 2.1386647 2.85378  ]
   [2.0937302 2.7649665 2.1433175 2.84821  ]]]], shape=(1, 3, 3, 4), dtype=float32)


As a sanity check, we can compare this against the outputs from individual heads we calculated earlier:

In [26]:
print("Per head outputs from using separate sets of weights per head:")
print(out0, "\n")
print(out1, "\n")
print(out2)

Per head outputs from using separate sets of weights per head:
tf.Tensor(
[[[3.1253777 3.0417163 2.3795164 5.062043 ]
  [3.0960062 3.0469806 2.3986073 5.0478077]
  [3.116835  3.0465755 2.3893042 5.0571923]]], shape=(1, 3, 4), dtype=float32) 

tf.Tensor(
[[[2.768069  3.8823671 2.5511048 2.6974177]
  [2.7745962 3.8707633 2.513121  2.666007 ]
  [2.767774  3.8762562 2.533748  2.6857417]]], shape=(1, 3, 4), dtype=float32) 

tf.Tensor(
[[[2.0942488 2.775998  2.155232  2.8350935]
  [2.0935578 2.7601702 2.1386647 2.85378  ]
  [2.0937302 2.7649665 2.1433175 2.84821  ]]], shape=(1, 3, 4), dtype=float32)


To get the final concatenated result, we need to reverse our **reshape** and **transpose** operation, starting with the **transpose** this time.

In [27]:
combined_out_b = tf.reshape(tf.transpose(all_heads_output, perm=[0, 2, 1, 3]), 
                            shape=(batch_size, seq_len, embed_dim))
print("Final output from using single query, key, value matrices:\n", 
      combined_out_b, "\n")
print("Final output from using separate query, key, value matrices per head:\n", 
      combined_out_a)

Final output from using single query, key, value matrices:
 tf.Tensor(
[[[3.1253777 3.0417163 2.3795164 5.062043  2.768069  3.8823671 2.5511048
   2.6974177 2.0942488 2.775998  2.155232  2.8350935]
  [3.0960062 3.0469806 2.3986073 5.0478077 2.7745962 3.8707633 2.513121
   2.666007  2.0935578 2.7601702 2.1386647 2.85378  ]
  [3.116835  3.0465755 2.3893042 5.0571923 2.767774  3.8762562 2.533748
   2.6857417 2.0937302 2.7649665 2.1433175 2.84821  ]]], shape=(1, 3, 12), dtype=float32) 

Final output from using separate query, key, value matrices per head:
 [[[3.1253777 3.0417163 2.3795164 5.062043  2.768069  3.8823671 2.5511048
   2.6974177 2.0942488 2.775998  2.155232  2.8350935]
  [3.0960062 3.0469806 2.3986073 5.0478077 2.7745962 3.8707633 2.513121
   2.666007  2.0935578 2.7601702 2.1386647 2.85378  ]
  [3.116835  3.0465755 2.3893042 5.0571923 2.767774  3.8762562 2.533748
   2.6857417 2.0937302 2.7649665 2.1433175 2.84821  ]]]


We can encapsulate everything we just covered in a class.

In [28]:
class MultiHeadSelfAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadSelfAttention, self).__init__()
    self.d_model = d_model
    self.num_heads = num_heads

    self.d_head = self.d_model // self.num_heads

    self.wq = tf.keras.layers.Dense(self.d_model)
    self.wk = tf.keras.layers.Dense(self.d_model)
    self.wv = tf.keras.layers.Dense(self.d_model)

    # Linear layer to generate the final output.
    self.dense = tf.keras.layers.Dense(self.d_model)
  
  def split_heads(self, x):
    batch_size = x.shape[0]

    split_inputs = tf.reshape(x, (batch_size, -1, self.num_heads, self.d_head))
    return tf.transpose(split_inputs, perm=[0, 2, 1, 3])
  
  def merge_heads(self, x):
    batch_size = x.shape[0]

    merged_inputs = tf.transpose(x, perm=[0, 2, 1, 3])
    return tf.reshape(merged_inputs, (batch_size, -1, self.d_model))

  def call(self, q, k, v, mask):
    qs = self.wq(q)
    ks = self.wk(k)
    vs = self.wv(v)

    qs = self.split_heads(qs)
    ks = self.split_heads(ks)
    vs = self.split_heads(vs)

    output, attn_weights = scaled_dot_product_attention(qs, ks, vs, mask)
    output = self.merge_heads(output)

    return self.dense(output), attn_weights


In [31]:
mhsa = MultiHeadSelfAttention(12, 3)

output, attn_weights = mhsa(x, x, x, None)
print(f"MHSA output{output.shape}:")
print(output)

MHSA output(1, 3, 12):
tf.Tensor(
[[[ 0.08030355 -0.2962575  -0.18756516  0.5291492  -0.42959774
    0.4763794   0.34621298 -0.3916294  -0.2647721  -0.00962536
   -0.0903991  -0.02550881]
  [ 0.08009857 -0.32823098 -0.1851799   0.53491    -0.4212141
    0.51921064  0.30727836 -0.40554237 -0.29415786 -0.02618809
   -0.0631731  -0.00486682]
  [ 0.12012464 -0.3080424  -0.1791061   0.53083223 -0.40426654
    0.55000114  0.3156727  -0.4015814  -0.29601887 -0.04674211
   -0.04817324  0.00603846]]], shape=(1, 3, 12), dtype=float32)


## Encoder Block

We can now build our **Encoder Block**. In addition to the **Multi-Head Self Attention** layer, the **Encoder Block** also has **skip connections**, **layer normalization steps**, and a **two-layer feed-forward neural network**. The original **Attention Is All You Need** paper also included some **dropout** applied to the self-attention output which isn't shown in the illustration below (see references for a link to the paper).

<div>
<img src="https://drive.google.com/uc?export=view&id=1D8sLDyQMqqhCjHWOn-I7rZKHugWxFyLy" width="500"/>
</div>

Since a two-layer feed forward neural network is used in multiple places in the transformer, here's a function which creates and returns one.

In [32]:
def feed_forward_network(d_model, hidden_dim):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(hidden_dim, activation='relu'),
      tf.keras.layers.Dense(d_model)
  ])

This is our encoder block containing all the layers and steps from the preceding illustration (plus dropout).

In [33]:
class EncoderBlock(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, hidden_dim, dropout_rate=0.1):
    super(EncoderBlock, self).__init__()

    self.mhsa = MultiHeadSelfAttention(d_model, num_heads)
    self.ffn = feed_forward_network(d_model, hidden_dim)

    self.dropout1 = tf.keras.layers.Dropout(dropout_rate)
    self.dropout2 = tf.keras.layers.Dropout(dropout_rate)

    self.layernorm1 = tf.keras.layers.LayerNormalization()
    self.layernorm2 = tf.keras.layers.LayerNormalization()
  
  def call(self, x, training, mask):
    mhsa_output, attn_weights = self.mhsa(x, x, x, mask)
    mhsa_output = self.dropout1(mhsa_output, training=training)
    mhsa_output = self.layernorm1(x + mhsa_output)

    ffn_output = self.ffn(mhsa_output)
    ffn_output = self.dropout2(ffn_output, training=training)
    output = self.layernorm2(mhsa_output + ffn_output)

    return output, attn_weights


Suppose we have an embedding dimension of 12, and we want 3 attention heads and a feed forward network with a hidden dimension of 48 (4x the embedding dimension). We would declare and use a single encoder block like so:

In [34]:
encoder_block = EncoderBlock(12, 3, 48)

block_output,  _ = encoder_block(x, True, None)
print(f"Output from single encoder block {block_output.shape}:")
print(block_output)

Output from single encoder block (1, 3, 12):
tf.Tensor(
[[[-1.5534191   0.7408428   0.25480387 -0.5420613  -0.6151438
    1.0893964   1.0193202  -0.9063178  -1.4228472   0.06072959
    1.7914821   0.08321424]
  [-2.0210521   0.7004914  -0.43833873 -1.2458332   0.6953257
    0.50829846  0.11558092 -0.07152531 -1.1905874   1.3490591
    1.2437296   0.35485134]
  [-1.8365915   1.2560222  -0.59348255 -1.1711762   0.41624412
    0.96616936  0.11589251 -0.3682211  -1.1482323   0.40115127
    1.5542463   0.40797755]]], shape=(1, 3, 12), dtype=float32)


## Word and Positional Embeddings

Let's now deal with the actual input to the **initial** encoder block. The inputs are going to be *positional word embeddings*. That is, word embeddings with some positional information added to them.
<br>

Let's start with **subword** tokenization. For demonstration, we'll use a subword tokenizer called **BPEmb**. It uses **Byte-Pair Encoding** and supports over two hundred languages. 

https://bpemb.h-its.org/


In [35]:
# Load the English tokenizer.
bpemb_en = BPEmb(lang="en")

downloading https://nlp.h-its.org/bpemb/en/en.wiki.bpe.vs10000.model


100%|██████████| 400869/400869 [00:00<00:00, 507878.85B/s]


downloading https://nlp.h-its.org/bpemb/en/en.wiki.bpe.vs10000.d100.w2v.bin.tar.gz


100%|██████████| 3784656/3784656 [00:01<00:00, 2700563.81B/s]


The library comes with embeddings for a number of words.

In [36]:
bpemb_vocab_size, bpemb_embed_size = bpemb_en.vectors.shape
print("Vocabulary size:", bpemb_vocab_size)
print("Embedding size:", bpemb_embed_size)

Vocabulary size: 10000
Embedding size: 100


In [37]:
# Embedding for the word "car".
bpemb_en.vectors[bpemb_en.words.index('car')]

array([-0.305548, -0.325598, -0.134716, -0.078735, -0.660545,  0.076211,
       -0.735487,  0.124533, -0.294402,  0.459688,  0.030137,  0.174041,
       -0.224223,  0.486189, -0.504649, -0.459699,  0.315747,  0.477885,
        0.091398,  0.427867,  0.016524, -0.076833, -0.899727,  0.493158,
       -0.022309, -0.422785, -0.154148,  0.204981,  0.379834,  0.070588,
        0.196073, -0.368222,  0.473406,  0.007409,  0.004303, -0.007823,
       -0.19103 , -0.202509,  0.109878, -0.224521, -0.35741 , -0.611633,
        0.329958, -0.212956, -0.497499, -0.393839, -0.130101, -0.216903,
       -0.105595, -0.076007, -0.483942, -0.139704, -0.161647,  0.136985,
        0.415363, -0.360143,  0.038601, -0.078804, -0.030421,  0.324129,
        0.223378, -0.523636, -0.048317, -0.032248, -0.117367,  0.470519,
        0.225816, -0.222065, -0.225007, -0.165904, -0.334389, -0.20157 ,
        0.572352, -0.268794,  0.301929, -0.005563,  0.387491,  0.261031,
       -0.11613 ,  0.074982, -0.008433,  0.259987, 

We don't need the embeddings since we're going to use our own embedding layer. What we're interested in are the subword tokens and their respective ids. The ids will be used as indexes into our embedding layer.<br>

If this doesn't sound familiar, refer to the module on word vectors:<br>
https://www.nlpdemystified.org/course/word-vectors

These are the subword tokens for our example sentence from the slides. **BPEmb** places underscores in front of any tokens which are whole words or intended to begin words.<br>

Remember that subword tokenizers are trained using count frequencies over a corpus. So these subword tokens are specific to **BPEmb**. Another subword tokenizer may output something different. This is why it's important that when we use a pretrained model, we make sure to use the pretrained model's tokenizer. We'll see this when we use pretrained transformers later in this module.

In [38]:
sample_sentence = "Where can I find a pizzeria?"
tokens = bpemb_en.encode(sample_sentence)
print(tokens)

['▁where', '▁can', '▁i', '▁find', '▁a', '▁p', 'iz', 'zer', 'ia', '?']


We can retrieve each subword token's respective id using the *encode_ids* method.

In [39]:
token_seq = np.array(bpemb_en.encode_ids("Where can I find a pizzeria?"))
print(token_seq)

[ 571  280  386 1934    4   24  248 4339  177 9967]


Now that we have a way to tokenize and vectorize sentences, we can declare and use an embedding layer with the same vocabulary size as **BPEmb** and a desired embedding size.

In [40]:
token_embed = tf.keras.layers.Embedding(bpemb_vocab_size, embed_dim)
token_embeddings = token_embed(token_seq)

# The untrained embeddings for our sample sentence.
print("Embeddings for: ", sample_sentence)
print(token_embeddings)

Embeddings for:  Where can I find a pizzeria?
tf.Tensor(
[[-0.02776699 -0.04904383  0.00416745  0.03465103  0.0311094   0.04116099
  -0.0316515   0.00955389  0.00087024  0.01622124  0.01202317  0.0063796 ]
 [ 0.02615741  0.005064   -0.02884558 -0.00879797 -0.00358888 -0.04069667
   0.03553255 -0.00771983  0.02414027  0.01274247 -0.00289358 -0.04980649]
 [ 0.00699419  0.00843186 -0.01811179  0.02979175 -0.02672805  0.03526754
   0.02984431 -0.00974999 -0.02727449  0.04638478 -0.04535105  0.02772393]
 [-0.04265243 -0.01685966 -0.00241258 -0.02178057  0.03070604 -0.01494353
  -0.03427645  0.02602315  0.01580607 -0.02898157  0.0428209   0.01457195]
 [-0.03940567 -0.03906695  0.01565515  0.03231524 -0.04281168  0.00685098
   0.02272804 -0.03537701  0.0001385  -0.01840727 -0.02663438  0.01647598]
 [-0.02686479 -0.0266047  -0.02222837  0.02929038 -0.03133326 -0.02990845
   0.00477933  0.03311404 -0.02108533 -0.01078187  0.00600853 -0.0145704 ]
 [ 0.03108071  0.02509686 -0.01303919 -0.0307078 

Next, we need to add *positional* information to each token embedding. As we covered in the slides, the original paper used sinusoidals but it's more common these days to just use another set of embeddings. We'll do the latter here.<br>

Here, we're declaring an embedding layer with rows equalling a maximum sequence length and columns equalling our token embedding size. We then generate a vector of position ids.

In [41]:
max_seq_len = 256
pos_embed = tf.keras.layers.Embedding(max_seq_len, embed_dim)

# Generate ids for each position of the token sequence.
pos_idx = tf.range(len(token_seq))
print(pos_idx)

tf.Tensor([0 1 2 3 4 5 6 7 8 9], shape=(10,), dtype=int32)


We'll use these position ids to index into the positional embedding layer.

In [42]:
# These are our positon embeddings.
position_embeddings = pos_embed(pos_idx)
print("Position embeddings for the input sequence\n", position_embeddings)

Position embeddings for the input sequence
 tf.Tensor(
[[ 0.02659403  0.03269235  0.04656676 -0.02572204  0.00252743  0.0460952
   0.03813703  0.04620485 -0.03946701  0.0146523   0.00726104  0.04377431]
 [ 0.02482054  0.03876705  0.00620854 -0.00561713  0.0129539   0.01198636
   0.04466181  0.01074811 -0.03739326  0.00631986  0.00503862  0.02549655]
 [-0.03971176 -0.04630612  0.04277991 -0.01652556  0.04950306 -0.03954357
   0.03758616 -0.03624535  0.04312929 -0.00795076  0.00344597 -0.027099  ]
 [ 0.00275147  0.0073516  -0.02497884  0.02825159 -0.00444879 -0.04665766
  -0.02186338 -0.00823791  0.03656547 -0.03324484 -0.04464153 -0.02185524]
 [ 0.00311292 -0.00594468 -0.03901132 -0.03837323  0.04708299 -0.01274588
   0.03940368  0.01327198  0.02984529  0.02660698 -0.01186252  0.04309386]
 [ 0.01468343  0.04341283 -0.04548661  0.04362733  0.04036992 -0.0192011
   0.01748762 -0.04088895  0.04747206  0.03696389 -0.02339031 -0.03218033]
 [-0.01998788 -0.02467362  0.01204836 -0.01139157  0.

The final step is to add our token and position embeddings. The result will be the input to the first encoder block.

In [43]:
input = token_embeddings + position_embeddings
print("Input to the initial encoder block:\n", input)

Input to the initial encoder block:
 tf.Tensor(
[[-0.00117296 -0.01635149  0.05073421  0.00892899  0.03363682  0.08725619
   0.00648553  0.05575874 -0.03859678  0.03087354  0.01928421  0.05015391]
 [ 0.05097795  0.04383105 -0.02263705 -0.0144151   0.00936503 -0.02871031
   0.08019435  0.00302828 -0.01325299  0.01906233  0.00214504 -0.02430994]
 [-0.03271757 -0.03787426  0.02466812  0.01326619  0.02277501 -0.00427603
   0.06743047 -0.04599533  0.0158548   0.03843402 -0.04190508  0.00062493]
 [-0.03990096 -0.00950806 -0.02739142  0.00647102  0.02625724 -0.06160119
  -0.05613983  0.01778524  0.05237154 -0.0622264  -0.00182064 -0.00728328]
 [-0.03629274 -0.04501163 -0.02335617 -0.00605799  0.00427131 -0.0058949
   0.06213172 -0.02210504  0.02998379  0.00819971 -0.0384969   0.05956984]
 [-0.01218136  0.01680813 -0.06771498  0.07291771  0.00903666 -0.04910956
   0.02226695 -0.00777491  0.02638673  0.02618202 -0.01738178 -0.04675074]
 [ 0.01109283  0.00042324 -0.00099083 -0.04209937  0.058144

## Encoder

Now that we have an encoder block and a way to embed our tokens with position information, we can create the **encoder** itself.<br>

Given a batch of vectorized sequences, the encoder creates positional embeddings, runs them through its encoder blocks, and returns contextualized tokens.

In [44]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, num_blocks, d_model, num_heads, hidden_dim, src_vocab_size,
               max_seq_len, dropout_rate=0.1):
    super(Encoder, self).__init__()

    self.d_model = d_model
    self.max_seq_len = max_seq_len

    self.token_embed = tf.keras.layers.Embedding(src_vocab_size, self.d_model)
    self.pos_embed = tf.keras.layers.Embedding(max_seq_len, self.d_model)

    # The original Attention Is All You Need paper applied dropout to the
    # input before feeding it to the first encoder block.
    self.dropout = tf.keras.layers.Dropout(dropout_rate)

    # Create encoder blocks.
    self.blocks = [EncoderBlock(self.d_model, num_heads, hidden_dim, dropout_rate) 
    for _ in range(num_blocks)]
  
  def call(self, input, training, mask):
    token_embeds = self.token_embed(input)

    # Generate position indices for a batch of input sequences.
    num_pos = input.shape[0] * self.max_seq_len
    pos_idx = np.resize(np.arange(self.max_seq_len), num_pos)
    pos_idx = np.reshape(pos_idx, input.shape)
    pos_embeds = self.pos_embed(pos_idx)

    x = self.dropout(token_embeds + pos_embeds, training=training)

    # Run input through successive encoder blocks.
    for block in self.blocks:
      x, weights = block(x, training, mask)

    return x, weights

If you're wondering about this code block here:


```
num_pos = input.shape[0] * self.max_seq_len
pos_idx = np.resize(np.arange(self.max_seq_len), num_pos)
pos_idx = np.reshape(pos_idx, input.shape)
pos_embeds = self.pos_embed(pos_idx)
```


This generates positional embeddings for a *batch* of input sequences. Suppose this was our batch of input sequences to the encoder.

In [45]:
# Batch of 3 sequences, each of length 10 (10 is also the 
# maximum sequence length in this case).
seqs = np.random.randint(0, 10000, size=(3, 10))
print(seqs.shape)
print(seqs)

(3, 10)
[[3240 8385 8402 6847 6541 2111 7022 9817 7865  884]
 [5893 1451 1595 1131 7494 1996 6031 2484 6844 3631]
 [ 213 2026 2668 1630 6915 7749 3392  821 7981 4772]]


We need to retrieve a positional embedding for every element in this batch. The first step is to create the respective positional ids...

In [46]:
pos_ids = np.resize(np.arange(seqs.shape[1]), seqs.shape[0] * seqs.shape[1])
print(pos_ids)

[0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6 7 8 9]


...and then reshape them to match the input batch dimensions.

In [47]:
pos_ids = np.reshape(pos_ids, (3, 10))
print(pos_ids.shape)
print(pos_ids)

(3, 10)
[[0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]]


We can now retrieve position embeddings for every token embedding.

In [48]:
pos_embed(pos_ids)

<tf.Tensor: shape=(3, 10, 12), dtype=float32, numpy=
array([[[ 0.02659403,  0.03269235,  0.04656676, -0.02572204,
          0.00252743,  0.0460952 ,  0.03813703,  0.04620485,
         -0.03946701,  0.0146523 ,  0.00726104,  0.04377431],
        [ 0.02482054,  0.03876705,  0.00620854, -0.00561713,
          0.0129539 ,  0.01198636,  0.04466181,  0.01074811,
         -0.03739326,  0.00631986,  0.00503862,  0.02549655],
        [-0.03971176, -0.04630612,  0.04277991, -0.01652556,
          0.04950306, -0.03954357,  0.03758616, -0.03624535,
          0.04312929, -0.00795076,  0.00344597, -0.027099  ],
        [ 0.00275147,  0.0073516 , -0.02497884,  0.02825159,
         -0.00444879, -0.04665766, -0.02186338, -0.00823791,
          0.03656547, -0.03324484, -0.04464153, -0.02185524],
        [ 0.00311292, -0.00594468, -0.03901132, -0.03837323,
          0.04708299, -0.01274588,  0.03940368,  0.01327198,
          0.02984529,  0.02660698, -0.01186252,  0.04309386],
        [ 0.01468343,  0.04

Let's try our encoder on a batch of sentences.

In [49]:
input_batch = [
    "Where can I find a pizzeria?",
    "Mass hysteria over listeria.",
    "I ain't no circle back girl."
]

bpemb_en.encode(input_batch)

[['▁where', '▁can', '▁i', '▁find', '▁a', '▁p', 'iz', 'zer', 'ia', '?'],
 ['▁mass', '▁hy', 'ster', 'ia', '▁over', '▁l', 'ister', 'ia', '.'],
 ['▁i', '▁a', 'in', "'", 't', '▁no', '▁circle', '▁back', '▁girl', '.']]

In [50]:
input_seqs = bpemb_en.encode_ids(input_batch)
print("Vectorized inputs:")
input_seqs

Vectorized inputs:


[[571, 280, 386, 1934, 4, 24, 248, 4339, 177, 9967],
 [1535, 1354, 1238, 177, 380, 43, 871, 177, 9935],
 [386, 4, 6, 9937, 9915, 467, 5410, 810, 3692, 9935]]

Note how the input sequences aren't the same length in this batch. In this case, we need to pad them out so that they are. If you're unfamiliar with why, refer to the notebook on Recurrent Neural Networks:<br>
https://colab.research.google.com/github/nitinpunjabi/nlp-demystified/blob/main/notebooks/nlpdemystified_recurrent_neural_networks.ipynb<br>

We'll do this using *pad_sequences*.<br>
https://www.tensorflow.org/api_docs/python/tf/keras/utils/pad_sequences

In [51]:
padded_input_seqs = tf.keras.preprocessing.sequence.pad_sequences(input_seqs, padding="post")
print("Input to the encoder:")
print(padded_input_seqs.shape)
print(padded_input_seqs)

Input to the encoder:
(3, 10)
[[ 571  280  386 1934    4   24  248 4339  177 9967]
 [1535 1354 1238  177  380   43  871  177 9935    0]
 [ 386    4    6 9937 9915  467 5410  810 3692 9935]]


Since our input now has padding, now's a good time to cover **masking**.
<br>

So given a mask, wherever there's a mask position set to 0, the corresponding position in the attention scores will be set to *-inf*. The resulting attention weight for the position will then be zero and no attending will occur for that position.
<br>

In the slides, we covered *look-ahead* masks for the decoder to prevent it from attending to future tokens, but we also need masks for padding.
<br>

In total, there are three masks involved:
1. The *encoder mask* to mask out any padding in the encoder sequences.

2. The *decoder mask* which is used in the decoder's **first** multi-head self-attention layer. It's a <u>combination of two masks</u>: one to account for the padding in target sequences, and the look-ahead mask.

3. The *memory mask* which is used in the decoder's **second** multi-head self-attention layer. The keys and values for this layer are going to be the encoder's output, and this mask will ensure the decoder doesn't attend to any encoder output which corresponds to padding. In practice, 1 and 3 are often the same.

The *scaled_dot_product_attention* function has this line:
```
  if mask is not None:
    scaled_scores = tf.where(mask==0, -np.inf, scaled_scores)
```

Let's create an encoder mask for our batch of input sequences.<br>

Wherever there's padding, we want the mask position set to zero.

In [52]:
enc_mask = tf.cast(tf.math.not_equal(padded_input_seqs, 0), tf.float32)
print("Input:")
print(padded_input_seqs, '\n')
print("Encoder mask:")
print(enc_mask)

Input:
[[ 571  280  386 1934    4   24  248 4339  177 9967]
 [1535 1354 1238  177  380   43  871  177 9935    0]
 [ 386    4    6 9937 9915  467 5410  810 3692 9935]] 

Encoder mask:
tf.Tensor(
[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]], shape=(3, 10), dtype=float32)


Keep in mind that the dimension of the attention matrix (for this example) is going to be:<br>
*(batch size, number of heads, query size, key size)*<br>
(3, 3, 10, 10)

So we need to expand the mask dimensions like so:

In [53]:
enc_mask = enc_mask[:, tf.newaxis, tf.newaxis, :]
enc_mask

<tf.Tensor: shape=(3, 1, 1, 10), dtype=float32, numpy=
array([[[[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]],


       [[[1., 1., 1., 1., 1., 1., 1., 1., 1., 0.]]],


       [[[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]]], dtype=float32)>

This way, the encoder mask will now be *broadcasted*.<br>
https://www.tensorflow.org/xla/broadcasting

Now we can declare an encoder and pass it batches of vectorized sequences.

In [54]:
num_encoder_blocks = 6

# d_model is the embedding dimension used throughout.
d_model = 12

num_heads = 3

# Feed-forward network hidden dimension width.
ffn_hidden_dim = 48

src_vocab_size = bpemb_vocab_size
max_input_seq_len = padded_input_seqs.shape[1]

encoder = Encoder(
    num_encoder_blocks,
    d_model,
    num_heads,
    ffn_hidden_dim,
    src_vocab_size,
    max_input_seq_len)

We can now pass our input sequences and mask to the encoder.

In [55]:
encoder_output, attn_weights = encoder(padded_input_seqs, training=True, 
                                       mask=enc_mask)
print(f"Encoder output {encoder_output.shape}:")
print(encoder_output)

Encoder output (3, 10, 12):
tf.Tensor(
[[[-9.81447279e-01  1.58553109e-01  3.08647305e-01  1.05245978e-01
   -4.97444898e-01 -1.10418642e+00  2.27815449e-01  2.07353091e+00
   -1.60608768e-01 -1.66570103e+00  1.50936115e+00  2.62344684e-02]
  [ 1.14114687e-01 -3.86174023e-01 -5.78241125e-02  1.10227728e+00
   -2.30436802e-01 -1.36967409e+00 -3.85039765e-03  2.19668126e+00
   -1.31332846e-02 -1.73358929e+00  8.14023972e-01 -4.32415068e-01]
  [ 4.20387238e-01 -2.92350620e-01 -3.98580670e-01 -1.84943840e-01
   -3.19586664e-01 -8.64325762e-01  4.60375994e-02  2.96577740e+00
    8.99892449e-02 -1.23597062e+00  2.89092094e-01 -5.15525520e-01]
  [-7.99383044e-01  1.19217634e+00 -1.28306836e-01 -5.87761521e-01
    3.67378622e-01 -4.71940130e-01 -6.08994424e-01  2.41246080e+00
    2.73611188e-01 -1.21747184e+00  6.11161828e-01 -1.04293108e+00]
  [-5.30392468e-01 -6.29925489e-01  1.14808536e+00 -6.95643190e-04
    7.35789657e-01  4.83751178e-01 -3.53797376e-01  1.97958100e+00
   -2.13648963e+00 

## Decoder Block

Let's build the **Decoder Block**. Everything we did to create the **encoder** block applies here. The major differences are that the **Decoder Block** has:
1. a **Multi-Head Cross-Attention** layer which uses the encoder's outputs as the keys and values.

2. an extra skip/residual connection along with an extra layer normalization step.

<div>
<img src="https://drive.google.com/uc?export=view&id=1WVT4SX49bnta4uscOTF4xrsxFI4PbPER" width="500"/>
</div>

In [56]:
class DecoderBlock(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, hidden_dim, dropout_rate=0.1):
    super(DecoderBlock, self).__init__()

    self.mhsa1 = MultiHeadSelfAttention(d_model, num_heads)
    self.mhsa2 = MultiHeadSelfAttention(d_model, num_heads)

    self.ffn = feed_forward_network(d_model, hidden_dim)

    self.dropout1 = tf.keras.layers.Dropout(dropout_rate)
    self.dropout2 = tf.keras.layers.Dropout(dropout_rate)
    self.dropout3 = tf.keras.layers.Dropout(dropout_rate)

    self.layernorm1 = tf.keras.layers.LayerNormalization()
    self.layernorm2 = tf.keras.layers.LayerNormalization()
    self.layernorm3 = tf.keras.layers.LayerNormalization()
  
  # Note the decoder block takes two masks. One for the first MHSA, another
  # for the second MHSA.
  def call(self, encoder_output, target, training, decoder_mask, memory_mask):
    mhsa_output1, attn_weights = self.mhsa1(target, target, target, decoder_mask)
    mhsa_output1 = self.dropout1(mhsa_output1, training=training)
    mhsa_output1 = self.layernorm1(mhsa_output1 + target)

    mhsa_output2, attn_weights = self.mhsa2(mhsa_output1, encoder_output, 
                                            encoder_output, 
                                            memory_mask)
    mhsa_output2 = self.dropout2(mhsa_output2, training=training)
    mhsa_output2 = self.layernorm2(mhsa_output2 + mhsa_output1)

    ffn_output = self.ffn(mhsa_output2)
    ffn_output = self.dropout3(ffn_output, training=training)
    output = self.layernorm3(ffn_output + mhsa_output2)

    return output, attn_weights


## Decoder

The decoder is almost the same as the encoder except it takes the encoder's output as part of its input, and it takes two masks: the decoder mask and memory mask.

In [57]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self, num_blocks, d_model, num_heads, hidden_dim, target_vocab_size,
               max_seq_len, dropout_rate=0.1):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.max_seq_len = max_seq_len

    self.token_embed = tf.keras.layers.Embedding(target_vocab_size, self.d_model)
    self.pos_embed = tf.keras.layers.Embedding(max_seq_len, self.d_model)

    self.dropout = tf.keras.layers.Dropout(dropout_rate)

    self.blocks = [DecoderBlock(self.d_model, num_heads, hidden_dim, dropout_rate) for _ in range(num_blocks)]

  def call(self, encoder_output, target, training, decoder_mask, memory_mask):
    token_embeds = self.token_embed(target)

    # Generate position indices.
    num_pos = target.shape[0] * self.max_seq_len
    pos_idx = np.resize(np.arange(self.max_seq_len), num_pos)
    pos_idx = np.reshape(pos_idx, target.shape)

    pos_embeds = self.pos_embed(pos_idx)

    x = self.dropout(token_embeds + pos_embeds, training=training)

    for block in self.blocks:
      x, weights = block(encoder_output, x, training, decoder_mask, memory_mask)

    return x, weights

Before we try the decoder, let's cover the masks involved. The decoder takes two masks:

The *decoder mask* which is a <u>combination of two masks</u>: one to account for the padding in target sequences, and the look-ahead mask. This mask is used in the decoder's **first** multi-head self-attention layer.

The *memory mask* which is used in the decoder's **second** multi-head self-attention. The keys and values for this layer are going to be the encoder's output, and this mask will ensure the decoder doesn't attend to any encoder output which corresponds to padding.

Suppose this is our batch of vectorized target *input* sequences for the decoder. These values are just made up.<br>

**Note**: If you need a refresher on how to prepare target input and output sequences for the decoder, refer to the [seq2seq notebook](https://colab.research.google.com/github/nitinpunjabi/nlp-demystified/blob/main/notebooks/nlpdemystified_seq2seq_and_attention.ipynb).



In [58]:
# Made up values.
target_input_seqs = [
    [1, 652, 723, 123, 62],
    [1, 25,  98, 129, 248, 215, 359, 249],
    [1, 2369, 1259, 125, 486],
]

As we did with the encoder input sequences, we need to pad out this batch so that all sequences within it are the same length.

In [59]:
padded_target_input_seqs = tf.keras.preprocessing.sequence.pad_sequences(target_input_seqs, padding="post")
print("Padded target inputs to the decoder:")
print(padded_target_input_seqs.shape)
print(padded_target_input_seqs)

Padded target inputs to the decoder:
(3, 8)
[[   1  652  723  123   62    0    0    0]
 [   1   25   98  129  248  215  359  249]
 [   1 2369 1259  125  486    0    0    0]]


We can create the padding mask the same way we did for the encoder.

In [60]:
dec_padding_mask = tf.cast(tf.math.not_equal(padded_target_input_seqs, 0), tf.float32)
dec_padding_mask = dec_padding_mask[:, tf.newaxis, tf.newaxis, :]
print(dec_padding_mask)

tf.Tensor(
[[[[1. 1. 1. 1. 1. 0. 0. 0.]]]


 [[[1. 1. 1. 1. 1. 1. 1. 1.]]]


 [[[1. 1. 1. 1. 1. 0. 0. 0.]]]], shape=(3, 1, 1, 8), dtype=float32)


As we covered in the slides, the look-ahead mask is a diagonal where the lower half are 1s and the upper half are zeros. This is easy to create using the *band_part* method:<br>
https://www.tensorflow.org/api_docs/python/tf/linalg/band_part

In [61]:
target_input_seq_len = padded_target_input_seqs.shape[1]
look_ahead_mask = tf.linalg.band_part(tf.ones((target_input_seq_len, 
                                               target_input_seq_len)), -1, 0)
print(look_ahead_mask)

tf.Tensor(
[[1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1.]], shape=(8, 8), dtype=float32)


To create the decoder mask, we just need to combine the padding and look-ahead masks. Note how the columns of the resulting decoder mask are all zero for padding positions.

In [62]:
dec_mask = tf.minimum(dec_padding_mask, look_ahead_mask)
print("The decoder mask:")
print(dec_mask)

The decoder mask:
tf.Tensor(
[[[[1. 0. 0. 0. 0. 0. 0. 0.]
   [1. 1. 0. 0. 0. 0. 0. 0.]
   [1. 1. 1. 0. 0. 0. 0. 0.]
   [1. 1. 1. 1. 0. 0. 0. 0.]
   [1. 1. 1. 1. 1. 0. 0. 0.]
   [1. 1. 1. 1. 1. 0. 0. 0.]
   [1. 1. 1. 1. 1. 0. 0. 0.]
   [1. 1. 1. 1. 1. 0. 0. 0.]]]


 [[[1. 0. 0. 0. 0. 0. 0. 0.]
   [1. 1. 0. 0. 0. 0. 0. 0.]
   [1. 1. 1. 0. 0. 0. 0. 0.]
   [1. 1. 1. 1. 0. 0. 0. 0.]
   [1. 1. 1. 1. 1. 0. 0. 0.]
   [1. 1. 1. 1. 1. 1. 0. 0.]
   [1. 1. 1. 1. 1. 1. 1. 0.]
   [1. 1. 1. 1. 1. 1. 1. 1.]]]


 [[[1. 0. 0. 0. 0. 0. 0. 0.]
   [1. 1. 0. 0. 0. 0. 0. 0.]
   [1. 1. 1. 0. 0. 0. 0. 0.]
   [1. 1. 1. 1. 0. 0. 0. 0.]
   [1. 1. 1. 1. 1. 0. 0. 0.]
   [1. 1. 1. 1. 1. 0. 0. 0.]
   [1. 1. 1. 1. 1. 0. 0. 0.]
   [1. 1. 1. 1. 1. 0. 0. 0.]]]], shape=(3, 1, 8, 8), dtype=float32)


We can now declare a decoder and pass it everything it needs. In our case, the *memory* mask is the same as the *encoder* mask.

In [63]:
decoder = Decoder(6, 12, 3, 48, 10000, 8)
decoder_output, _ = decoder(encoder_output, padded_target_input_seqs, 
                            True, dec_mask, enc_mask)
print(f"Decoder output {decoder_output.shape}:")
print(decoder_output)

Decoder output (3, 8, 12):
tf.Tensor(
[[[-8.20376039e-01 -2.25071348e-02 -1.73389173e+00  5.14293849e-01
    7.83088923e-01  4.92077500e-01  1.41714096e+00  1.58942401e+00
   -1.58164060e+00 -2.18044713e-01 -3.12101364e-01 -1.07464015e-01]
  [-9.57718790e-01  1.09101617e+00 -1.41629076e+00  1.30101240e+00
    9.46319640e-01 -4.32391614e-01  1.45061743e+00 -3.26657921e-01
   -1.44761622e+00 -7.27980971e-01  5.19258727e-04  5.19171476e-01]
  [-6.51479542e-01  6.75288558e-01 -9.94457841e-01  1.61265159e+00
    8.21617305e-01 -5.26398383e-02  1.20156229e+00 -3.88593018e-01
   -1.84903026e+00 -7.81689405e-01 -5.51537395e-01  9.58307505e-01]
  [-1.05034578e+00  1.50975800e+00 -9.84553039e-01  5.48390687e-01
    1.45186591e+00 -5.78179538e-01  1.16076660e+00  3.55888039e-01
   -1.44815671e+00 -1.10243905e+00 -1.56118512e-01  2.93123275e-01]
  [-9.23666596e-01  6.09717011e-01 -3.46251160e-01  2.30735278e+00
    2.36855686e-01  6.13248311e-02  3.10473084e-01 -1.06656981e+00
   -1.39959121e+00 -

## Transformer

We now have all the pieces to build the **Transformer** itself, and it's pretty simple. 

In [64]:
class Transformer(tf.keras.Model):
  def __init__(self, num_blocks, d_model, num_heads, hidden_dim, source_vocab_size,
               target_vocab_size, max_input_len, max_target_len, dropout_rate=0.1):
    super(Transformer, self).__init__()

    self.encoder = Encoder(num_blocks, d_model, num_heads, hidden_dim, source_vocab_size, 
                           max_input_len, dropout_rate)
    
    self.decoder = Decoder(num_blocks, d_model, num_heads, hidden_dim, target_vocab_size,
                           max_target_len, dropout_rate)
    
    # The final dense layer to generate logits from the decoder output.
    self.output_layer = tf.keras.layers.Dense(target_vocab_size)

  def call(self, input_seqs, target_input_seqs, training, encoder_mask,
           decoder_mask, memory_mask):
    encoder_output, encoder_attn_weights = self.encoder(input_seqs, 
                                                        training, encoder_mask)

    decoder_output, decoder_attn_weights = self.decoder(encoder_output, 
                                                        target_input_seqs, training,
                                                        decoder_mask, memory_mask)

    return self.output_layer(decoder_output), encoder_attn_weights, decoder_attn_weights


In [65]:
transformer = Transformer(
    num_blocks = 6,
    d_model = 12,
    num_heads = 3,
    hidden_dim = 48,
    source_vocab_size = bpemb_vocab_size,
    target_vocab_size = 7000, # made-up target vocab size.
    max_input_len = padded_input_seqs.shape[1],
    max_target_len = padded_target_input_seqs.shape[1])

transformer_output, _, _ = transformer(padded_input_seqs, 
                                       padded_target_input_seqs, True, 
                                       enc_mask, dec_mask, memory_mask=enc_mask)
print(f"Transformer output {transformer_output.shape}:")
print(transformer_output) # If training, we would use this output to calculate losses.

Transformer output (3, 8, 7000):
tf.Tensor(
[[[-0.07941675  0.05409476  0.03010898 ...  0.05203716  0.00357621
    0.05687221]
  [-0.13604656  0.06108982  0.0322881  ...  0.0856856   0.05374376
    0.0136425 ]
  [-0.11966557  0.09301373 -0.00927811 ...  0.03227054  0.03134258
    0.04105105]
  ...
  [-0.12380788  0.08860358 -0.00911726 ...  0.03578832  0.04158672
    0.04338367]
  [-0.12591834  0.0780643  -0.01962833 ...  0.0590167   0.04727542
    0.04294289]
  [-0.12284482  0.09355265 -0.00717188 ...  0.02926615  0.06409989
    0.03331104]]

 [[-0.1028582   0.09806318  0.00062626 ...  0.08036139  0.0601952
    0.01978529]
  [-0.11430392  0.12440582 -0.00886312 ... -0.01226565  0.06976765
    0.05570981]
  [-0.12426215  0.0874722   0.02707232 ...  0.04963186  0.06013392
    0.04100896]
  ...
  [-0.1419553   0.09015933  0.01407075 ...  0.00616651  0.05506806
    0.03804151]
  [-0.14407815  0.05857605  0.08097555 ...  0.0598792   0.02123609
    0.06153494]
  [-0.14566334  0.06410029  0.

That's the whole original transformer from scratch. From here, if you want to train this transformer, you can use the same approach we used when we built the translation model with attention in the [seq2seq notebook](https://colab.research.google.com/github/nitinpunjabi/nlp-demystified/blob/main/notebooks/nlpdemystified_seq2seq_and_attention.ipynb#scrollTo=x8Ef_eWXjWMn&line=3&uniqifier=1). Remember to use a learning rate warmup (Refer to the paper for more information on this).

It's useful to know how these models work under the hood, but to train our own transformer to get impressive results is expensive. Both in terms of compute and data.<br>

Fortunately, there's a zoo of **pretrained** transformer models we can use. We'll explore that next.

# Pre-Training and Transfer Learning with Hugging Face and OpenAI

**IMPORTANT**<br>
Enable **GPU acceleration** by going to *Runtime > Change Runtime Type*. Keep in mind that, on certain tiers, you're not guaranteed GPU access depending on usage history and current load.
<br><br>
Also, if you're running this in the cloud rather than a local Jupyter server on your machine, then the notebook will *timeout* after a period of inactivity.
<br><br>
Refer to this link on how to run Colab notebooks locally on your machine to avoid this issue:<br>
https://research.google.com/colaboratory/local-runtimes.html

We'll explore pre-training and transfer learning using the **Transformers** library from [Hugging Face](https://huggingface.co/). **Transformers** is an API and toolkit to download pre-trained models and further train them as needed. <br>

We'll start with the **pipelines** module which abstracts a lot of operations such as tokenization, vectorization, inference, etc.<br>

With **Transformers pipelines**, we can just feed text input and get text output. And there are **pipelines** for common tasks including classification, NER, summarization, etc.<br>
https://huggingface.co/docs/transformers/index<br>
https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#pipelines

To get started, we'll need to install **Transformers**.

In [ ]:
!pip install transformers
!pip install datasets

In [ ]:
import operator
import pandas as pd
import tensorflow as tf
import transformers

from datasets import load_dataset
from tensorflow import keras
from transformers import AutoTokenizer
from transformers import pipeline
from transformers import TFAutoModelForQuestionAnswering

## Getting up and running quickly with Hugging Face Pipelines

We'll use the **pipeline** (note the singular) abstraction which wraps all the other pipelines. Put simply, it'll be our interface to doing a bunch of NLP tasks.

Using the **pipeline** abstraction is easy. We can instantiate a pipeline with a particular task, and it'll automatically download a suitable tokenizer and model behind the scenes for us and take care of the input and output operations.<br>
https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.pipeline<br>



Here, we're retrieving a pipeline for text-classification.

In [ ]:
classifier = pipeline("text-classification")

Note the warning message about how no model was supplied. When we instantiate a pipeline for a task without specifying a particular model to perform the task, **Transformers** uses a default model. This is good enough for prototyping but for production, we'll want to specify which model to use for the task since the default can change. We'll see how to do this further below.

We can use the pipeline immediately to classify some text. Tokenization, vectorization, etc is taken care of behind the scenes.

In [ ]:
classifier("Alice was excited to go the island but it didn't live up to the hype.")

In [ ]:
classifier("Bob doesn't do well in group situations but he said it wasn't bad.")

There's support for summarization...

In [ ]:
summarizer = pipeline("summarization")

In [ ]:
text = """
Hans Niemann is launching a counterattack in his dispute with chess world 
champion Magnus Carlsen, filing a federal lawsuit that accuses Carlsen of 
maliciously colluding with others to defame the 19-year-old grandmaster and 
ruin his career.

It's the latest move in a scandal that has injected unprecedented levels of 
drama into the world of elite chess since early September, when Carlsen 
suggested Niemann's upset victory over him at the Sinquefield Cup tournament 
in St. Louis was the result of cheating.

Niemann wants a federal court in Missouri's eastern district to award him at 
least $100 million in damages. Defendants in the lawsuit include Carlsen, his 
company Play Magnus Group, the online platform Chess.com and its leader, Danny 
Rensch, along with grandmaster Hikaru Nakamura.
"""

In [ ]:
summarizer(text)

...and question answering (extractive in this example).

In [ ]:
qa = pipeline("question-answering")

In [ ]:
context="""
Hugging Face was founded in 2016 by Clément Delangue, Julien Chaumond, and 
Thomas Wolf originally as a company that developed a chatbot app targeted at 
teenagers.[2] After open-sourcing the model behind the chatbot, the company 
pivoted to focus on being a platform for democratizing machine learning. In March 
2021, Hugging Face raised $40 million in a Series B funding round.
"""

question = "Who are the Hugging Face founders?"

qa(question=question, context=context)

Extractive question-answering models work fine for certain domains, document structures, and questions. But situations that require reasoning, more complex parsing, or contain ambiguity can trip it up.

In [ ]:
question = "What does Hugging Face do?"
qa(question=question, context=context)

There are ready-made pipelines for a number of tasks:<br>
https://huggingface.co/docs/transformers/main/en/quicktour#pipeline

Let's say we want a pipeline that uses a particular model. On the Hugging Face model hub, you'll find both pre-trained models (e.g. BERT) *and* pre-trained models that have been fine-tuned for all sorts of tasks (e.g. BERT for text classification). These models are contributed by Hugging Face, other companies, institutions, and individuals. You can (and are encouraged) to train or fine-tune a model and upload it for others to use.<br>
https://huggingface.co/models
<br><br>
For example, here's a collection of pre-trained models that have been tuned for text classification.<br>
https://huggingface.co/models?pipeline_tag=text-classification&sort=downloads
<br><br>
This particular one is a pre-trained *Roberta-base* model that's been fine-tuned on Twitter data for sentiment analysis:
https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment
<br><br>
Note how you can try the model directly on the model page.

Let's say we want to download and use a particular model. For example, this *BERT-base* model fine-tuned for NER:
https://huggingface.co/dslim/bert-base-NER
<br>

We just need to pass the model path during pipeline instantiation.

In [ ]:
ner = pipeline(model="dslim/bert-base-NER")

In [ ]:
text = "Panic ensues in Redmond as love child of Microsoft and OpenAI declares humanity obsolete." 
ner(text)

The **Transformers** library provides a bunch of helper classes to help with training models. And beyond the model hub, Hugging Face also hosts datasets, provides *spaces* where you can host your app, and offers a bunch of services such as cloud hardware and inference endpoints to help deploy your model.<br>
Datasets: https://huggingface.co/datasets<br>
Spaces: https://huggingface.co/spaces<br>

With Hugging Face, you can build an ML app prototype within minutes and iterate quickly from there.<br>
https://huggingface.co/docs<br>

Learn more about how to build with Hugging Face through their free course and fantastic book:<br>
https://huggingface.co/course<br>
https://www.oreilly.com/library/view/natural-language-processing/9781098136789/


## Fine-Tuning a Pre-Trained Model.

Let's say the model hub doesn't have a model that exactly suits your purpose. Perhaps you work in a particular domain and need to fine-tune a model using your own dataset.<br>

In this section, we'll walk through how to download a pre-trained model and fine-tune it. Our example covers extractive question answering but it's the same idea with other tasks.

We'll fine-tune using a dataset from the **Datasets** hub.<br>
https://huggingface.co/datasets<br><br>
Hugging Face provides a **datasets** library to download and interact with the datasets. It's similar to the Tensorflow Dataset library we used in that it can hold data and provides a bunch of methods to preprocess that data.<br>
https://huggingface.co/docs/datasets/ndex

The **Datasets** hub holds a bunch of question answering datasets.<br>
https://huggingface.co/datasets?task_categories=task_categories:question-answering&sort=downloads<br>


They differ based on data source, domain, and level of challenge. Since we're in a constrained environment (Colab free tier) and just learning how to fine-tune, we'll use SQuAD, a famous dataset comprised of crowd-sourced questions on a set of Wikipedia articles, and where the answer is a span of text in the article.<br>
https://huggingface.co/datasets/squad


In [ ]:
data = load_dataset("squad")

The **datasets** library downloads and automatically splits the data into train and validation sets. It returns a dictionary of **Dataset** objects:<br>
https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.DatasetDict<br>
https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset<br>


A **Dataset** object wraps an Apache Arrow table and provides a bunch of helper functions on top of it.<br>
https://arrow.apache.org/

In [ ]:
data

Glaning at the data, we see every context (Wikipedia passage) is used multiple times. i.e., there are multiple questions and answers for each context.<br>

Every answer is a span of text from the context and the character position where the answer starts in the context is given.

In [ ]:
pd.DataFrame(data['train'][0, 1, 2, 100, 101, 102], 
             columns=["context", "question", "answers"])

Here's what we need to do:
1. Choose a pre-trained model based on what we want to accomplish and our constraints.
2. Download the appropriate tokenizer for the pre-trained model.
3. Tokenize and vectorize our dataset.
4. Mark where each answer starts and ends in our vectorized dataset.
5. Download the pre-trained model.
6. Fine-tune the pre-trained model with the vectorized dataset.

Given the free tier of Colab doesn't have a lot of GPU memory and that we're just trying to fine-tune a simple, extractive question answering model, we'll use *distilroberta-base*.<br>
https://huggingface.co/distilroberta-base
<br><br>
Recall from the slides that *DistilBERT* was created using a technique called *knowledge distillation*. The result is a model that performs almost as well as BERT but is 40% smaller and 60% faster.<br>
DistilBert Paper: https://arxiv.org/abs/1910.01108<br>
https://en.wikipedia.org/wiki/Knowledge_distillation
<br><br>
*distilroberta-base* was created by applying knowledge distillation to *Roberta-Base*, a more powerful model than BERT.<br>
Roberta paper: https://arxiv.org/abs/1907.11692<br><br>
The **Transformers** library provides a set of Auto Classes that can automatically retrieve configurations, tokenizers, and models based on a path or a name. We'll use the **AutoTokenizer** class to get the right tokenizer for *distilroberta-base*.<br>
https://huggingface.co/docs/transformers/main/en/model_doc/auto<br>
https://huggingface.co/docs/transformers/main/en/model_doc/auto#transformers.AutoTokenizer



In [ ]:
model_name = 'distilroberta-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)

Calling *encode* converts a string to a sequence of integer token ids.<br>
https://huggingface.co/docs/transformers/main/en/internal/tokenization_utils#transformers.PreTrainedTokenizerBase.encode

In [ ]:
t = "Where can I find a pizzeria?"
print(tokenizer.encode(t))

But to tokenize, we call the tokenizer object directly (i.e. using *\_\_call\_\_*).<br>
https://huggingface.co/docs/transformers/main/en/internal/tokenization_utils#transformers.PreTrainedTokenizerBase.__call__<br>

This returns a sequence of ids and an attention mask in a **BatchEncoding** object:<br>
https://huggingface.co/docs/transformers/main/en/glossary#input-ids<br>
https://huggingface.co/docs/transformers/main/en/main_classes/tokenizer#transformers.BatchEncoding<br>

Since there's no padding on this sample string, the mask is all 1s.

In [ ]:
encoded_t = tokenizer(t)
print(encoded_t)

We can convert the ids back to tokens using *convert_ids_to_tokens*.<br>
https://huggingface.co/docs/transformers/main/en/main_classes/tokenizer#transformers.PreTrainedTokenizer.convert_ids_to_tokens<br>

Note how the tokenizer added a start of sequence token (\<s\>), end of sequence token (\</s\>), and how it uses Ġ to signal a word has preceding whitespace. Keep in mind that what you're seeing here is the output from the *distilroberta-base* tokenizer. Other tokenizers may work differently.

In [ ]:
print(tokenizer.convert_ids_to_tokens(encoded_t['input_ids']))

As we covered in the slides, for question answering, we need to encode the question and context as a pair. In our case, we can do that by passing in both strings separated by a comma.

In [ ]:
encoded_pair = tokenizer("this is a question", "this is the context")
print(encoded_pair)

The *distilroberta-base* tokenizer uses a double \</s\>\</s\> as a separator.

In [ ]:
print(tokenizer.convert_ids_to_tokens(encoded_pair['input_ids']))

**Side note**:<br>
Most of the tokenizers in the **Transformers** library come in two versions: a Python implementation and a faster Rust implementation. When available, **Autotokenizer** will download the fast version.<br>
https://huggingface.co/docs/transformers/main_classes/tokenizer<br>

We can check whether we have a fast tokenizer.


In [ ]:
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

Suppose we tokenize this question/context pair...

In [ ]:
context = "Sarah went to The Mirthless Cafe last night to meet her friend."
question = "Where did Sarah go?"

# The answer span and the answer's starting character position in the context.
answer = "The Mirthless Cafe"
answer_start = 14

In [ ]:
x = tokenizer(question, context)
x

Note how the word *Mirthless* gets tokenized into subwords. For legibility, we're using *batch_decode* to convert the input_ids to strings.<br>
https://huggingface.co/docs/transformers/v4.23.1/en/internal/tokenization_utils#transformers.PreTrainedTokenizerBase.batch_decode

In [ ]:
tokenizer.batch_decode(x['input_ids'])

When we tokenize our dataset, there will probably be question/context pairs which exceed our model's maximum sequence length. In *Roberta*'s case, that's 512. Available GPU memory may make us further reduce the maximum sequence length of our input.<br>

Let's say the maximum sequence length we can handle is 15, so we truncate the context.

In [ ]:
example_max_length = 15
x = tokenizer(question, context, max_length=example_max_length, 
              truncation="only_second")
x

The problem here is that the answer span gets chopped off by truncation. In other situations, the answer may not be included at all.

In [ ]:
tokenizer.batch_decode(x['input_ids'])

To ensure we tokenize all context tokens while respecting a maximum length, we can set *return_overflowing_tokens* to **True**. The end effect is to split the input into multiple question/context sequences, with each context sequence being a continuation of the previous one. Since the last one may be shorter than the max length, we set the right padding length as well.<br>

What we get back are multiple *input_id* sequences.

In [ ]:
x = tokenizer(question, context, max_length=example_max_length, 
              truncation="only_second", return_overflowing_tokens=True, 
              padding="max_length")
x

In [ ]:
len(x['input_ids'])

Looking at the decoded sequences, we see the entire context is included across three sequences (along with padding on the last one).<br>

In [ ]:
tokenizer.batch_decode(x['input_ids'])

Note a few things from the encoded object *x*:
- The last *attention_mask* sequence has 0s to signify padding.
- The *overflow_to_sample_mapping* array tells us which question/context pair each *input_ids* sequence comes from. In our example, we tokenized a single question/context pair which resulted in three *input_ids* sequences, so *overflow_to_sample_mapping* is 3 0s.<br>

If we tokenize two question/context pairs, we'll see the *overflow_to_sample_mapping* reflect that.

In [ ]:
tokenizer(['question 1', 'question 2'], 
          ['context 1', 'context 2'], 
          return_overflowing_tokens=True)

But there's still a problem here in that none of the sequences contain the full answer ("The Mirthless Cafe"). Right now, the correct full answer is split across sequences.<br>

To counter this, we can tokenize our question/context pair into overlapping sequences by setting a *stride* length. We did something similar when we prepared the dataset for our [character-level language model](https://colab.research.google.com/github/nitinpunjabi/nlp-demystified/blob/main/notebooks/nlpdemystified_recurrent_neural_networks.ipynb#scrollTo=X1c-ihOByy88).

In [ ]:
stride = 5
x = tokenizer(question, context, max_length=example_max_length, 
              truncation="only_second", return_overflowing_tokens=True,
              stride=stride, padding="max_length")

By setting a stride of 5, each context sequence starts 5 subwords back from the previous sequence.<br>

This way, two of our tokenized sequences now contain the full answer.


In [ ]:
tokenizer.batch_decode(x['input_ids'])

We now have a way to tokenize our question/context pairs.<br>

Our tokenizer returned this **BatchEncoding** object:

In [ ]:
print(x.keys(), '\n')
x

To fine-tune a model for question answering, our pre-trained *distilroberta-base* model expects this object to contain two more pieces of information:
- *start_positions*: the token positions where answers begin.
- *end_positions*: the token positions where answers end.<br>

https://huggingface.co/docs/transformers/main/en/model_doc/roberta#transformers.RobertaForQuestionAnswering.forward

All we have in our example (and the SQuAD dataset) is the position of the starting  <u>character</u> of the answer.

In [ ]:
print(answer_start)
print(context[answer_start:answer_start+len(answer)])

We need to use this to locate the <u>token</u> positions where each answer starts and ends in every *input_ids* sequence. In some cases, the complete answer may not be in a particular sequence. We need to handle those cases as well.<br>

To do this, we'll get more information by setting *return_offsets_mapping* to **True** in the tokenizer.

In [ ]:
x = tokenizer(question, context, max_length=example_max_length, 
              truncation="only_second", return_overflowing_tokens=True,
              stride=stride, return_offsets_mapping=True,
              padding="max_length")
x

This results in *offset_mapping* sequences, one for each *input_ids* sequence. 

In [ ]:
print(len(x['input_ids']))
print(len(x['offset_mapping']))

Each entry in an *offset_mapping* tells us the starting and ending character position of each token in the original string. An offset mapping of (0,0) represents a special token (e.g. \<s\>).<br>

For example, here's the first *input_ids* sequence along with its respective *offset_mapping*.

In [ ]:
print(x['input_ids'][0])
print(x['offset_mapping'][0])

If we convert the first non-special input id to a token, and use the first non-special offset_mapping to extract a span from the question string, we get a match.

In [ ]:
print("First non-special input_id converted to token:")
print(tokenizer.convert_ids_to_tokens(x['input_ids'][0][1]), "\n")

offset = x['offset_mapping'][0][1]
print(f"Span extracted from context using corresponding offset_mapping {offset}:")
print(question[offset[0]:offset[1]])

Since we know the character position of where the answer starts, we can use that and *offset_mapping* to get the start and ending token positions of the answer span.<br>

The only remaining issue is identifying whether an offset is for a question or a context. Looking at the first two *offset_mappings*, note that:<br>
1. In the first sequence, both the question and context *offset_mappings* start from zero.
2. In the second sequence, the context *offset_mapping* values carry on from the previous sequence (after accounting in the stride).

In [ ]:
print(x['offset_mapping'][0])
print(x['offset_mapping'][1])

This means we need to identify
1. which *offset_mappings* belong to a context.
2. whether a particular sequence contains the answer at all.<br>

The first can be done using the *sequence_ids* method on the encoding object. Each *input_ids* sequence has a corresponding *sequence_ids* list which tells us whether a token is part of a question, part of a context, or a special token.<br>
https://huggingface.co/docs/transformers/main/en/main_classes/tokenizer#transformers.BatchEncoding.sequence_ids

In [ ]:
print(x['input_ids'][0])
print(x.sequence_ids(0))

So to identify whether a token is part a context, we can use *sequence_ids* to check whether a token position maps to 1.

For the second issue, we can check whether the answer start and end character positions are within the lowest and highest offset mapping values respectively.

In [ ]:
# We can calculate the answer end character position using the answer length.
answer_end = answer_start + len(answer)

print("Answer start character position:", answer_start)
print("Answer end character position:", answer_end)
print("Answer pulled from context:", context[answer_start:answer_end])

Let's find the start and end token positions from our collection of sequences. The full answer is not in the first sequence, but is in the third sequence. So let's experiment with those.

In [ ]:
tokenizer.batch_decode(x['input_ids'])

First get all the information we need for the first sequence.

In [ ]:
input_ids = x['input_ids'][0]
offset_mapping = x['offset_mapping'][0]
seq_ids = x.sequence_ids(0)

Determine where the context tokens start and end in the sequence.

In [ ]:
# These are the sequence ids
print("Sequence IDs: ", seq_ids)

In [ ]:
# Get the start index position (i.e. the first occurrence of 1).
context_pos_start = seq_ids.index(1)

In [ ]:
# Utility function to find the *last* occurrence of a sequence.
def rindex(lst, value):
    return len(lst) - operator.indexOf(reversed(lst), value) - 1

# Get the end index position (i.e. the last occurrence of 1).
context_pos_end = rindex(seq_ids, 1)

In [ ]:
print("Context tokens begin at position", context_pos_start)
print("Context tokens end at position", context_pos_end)

Now that we know which tokens are part of the context, we can look at their corresponding offset mappings to check whether the start and end character positions are within the offsets.

In [ ]:
# These are the corresponding offsets.
context_offsets = offset_mapping[context_pos_start:context_pos_end+1]
print(context_offsets)

In [ ]:
print("Is the lowest offset value lower than or equal to the starting character position?")
print("Answer starting character position:", answer_start)
print("First offset:", context_offsets[0])

# Note how we're checking the first tuple value.
print(context_offsets[0][0] <= answer_start)

In [ ]:
print("Is the highest offset value higher than or equal to the ending character position?")
print("Answer ending character position:", answer_end)
print("Last offset:", context_offsets[-1])

# Note how how we're checking the second tuple value.
print(context_offsets[-1][1] >= answer_end)

So the first sequence contains a part of the answer but the full answer gets truncated. This matches a visual inspection:

In [ ]:
print(tokenizer.batch_decode(input_ids))

Let's now do the same with the third sequence.

In [ ]:
input_ids = x['input_ids'][2]
offset_mapping = x['offset_mapping'][2]
seq_ids = x.sequence_ids(2)

context_pos_start = seq_ids.index(1)
context_pos_end = rindex(seq_ids, 1)

context_offsets = offset_mapping[context_pos_start:context_pos_end+1]

print("Is the lowest offset value lower than or equal to the starting character position?")
print("Answer starting character position:", answer_start)
print("First offset:", context_offsets[0])

# Note how we're checking the first tuple value.
print(context_offsets[0][0] <= answer_start)

print("Is the highest offset value higher than or equal to the ending character position?")
print("Answer ending character position:", answer_end)
print("Last offset:", context_offsets[-1])

# Note how how we're checking the second tuple value.
print(context_offsets[-1][1] >= answer_end)


Now that we've confirmed the third sequence contains the full answer, we need to identify where the answer starts and ends in the *input_ids*. We can do this by scanning the offset_mapping from the left to find the start, and from the right to find the end.

In [ ]:
s = e = 0

# Start scanning the offset_mapping from the
# left to find the token position where the answer starts.
# It's not guaranteed a tokenizer will output a token where the
# starting character matches the first answer character. When
# this happens, we take the previous token's position as our start.
i = context_pos_start
while offset_mapping[i][0] < answer_start:
  i += 1
if offset_mapping[i][0] == answer_start:
  s = i
else:
  s = i - 1

# Same idea when finding the ending token position.
j = context_pos_end
while offset_mapping[j][1] > answer_end:
  j -= 1      
if offset_mapping[j][1] == answer_end:
  e = j
else:
  e = j + 1

In [ ]:
print("Answer start token position in context:", s)
print("Answer end token position in context:", e)

In [ ]:
print("Answer lifted from context:")
tokenizer.batch_decode(input_ids[s:e+1])

All the logic we stepped through so far is encapsulated in the following method. We'll use this to process our dataset.

In [ ]:
def prepare_dataset(examples):
  # Some tokenizers don't strip spaces. If there happens to be question text
  # with excessive spaces, the context may not get encoded at all.
  examples["question"] = [q.lstrip() for q in examples["question"]]
  examples["context"] = [c.lstrip() for c in examples["context"]]

  # Tokenize.  
  tokenized_examples = tokenizer(
      examples['question'],
      examples['context'],
      truncation="only_second",
      max_length = max_length,
      stride=stride,
      return_overflowing_tokens=True,
      return_offsets_mapping=True,
      padding="max_length"
  )

  # We'll collect a list of starting positions and ending positions.
  tokenized_examples['start_positions'] = []
  tokenized_examples['end_positions'] = []

  # Work through every sequence.
  for seq_idx in range(len(tokenized_examples['input_ids'])):
    seq_ids = tokenized_examples.sequence_ids(seq_idx)
    offset_mappings = tokenized_examples['offset_mapping'][seq_idx]

    cur_example_idx = tokenized_examples['overflow_to_sample_mapping'][seq_idx]
    answer = examples['answers'][cur_example_idx]
    answer_text = answer['text'][0]
    answer_start = answer['answer_start'][0]
    answer_end = answer_start + len(answer_text)

    context_pos_start = seq_ids.index(1)
    context_pos_end = rindex(seq_ids, 1)

    s = e = 0
    if (offset_mappings[context_pos_start][0] <= answer_start and
        offset_mappings[context_pos_end][1] >= answer_end):
      i = context_pos_start
      while offset_mappings[i][0] < answer_start:
        i += 1
      if offset_mappings[i][0] == answer_start:
        s = i
      else:
        s = i - 1

      j = context_pos_end
      while offset_mappings[j][1] > answer_end:
        j -= 1      
      if offset_mappings[j][1] == answer_end:
        e = j
      else:
        e = j + 1

    tokenized_examples['start_positions'].append(s)
    tokenized_examples['end_positions'].append(e)

  return tokenized_examples

Before we process, we'll set maximum sequence length, stride, and batch size values.<br>

I arrived at these values through experimentation. Even though *distilroberta-base* has a maximum sequence length of 512, using the full capacity (or a large batch value) results in an out-of-memory error while the attention scores are being calculated. This is on Colab's free tier. On the premium tier, you can use larger sequence lengths or batch values.<br>

The nature of the data will also influence the values.

In [ ]:
max_length = 400
stride = 100
batch_size = 32

We can map over the **Dataset** objects and apply our prepare method to the examples in batches.<br>
https://huggingface.co/docs/datasets/main/en/nlp_process<br>
https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.map<br>

*remove_columns* removes the original data columns and leaves only the post-tokenization columns in place. We can also parallelize processing by using the *num_proc* parameter.<br>
https://huggingface.co/docs/datasets/main/en/process#multiprocessing

In [ ]:
tokenized_datasets = data.map(
  prepare_dataset,
  batched=True,
  remove_columns=data["train"].column_names,
  num_proc=2,
)

Our tokenized dataset still contains two entries (*offset_mapping* and *overflow_to_sample_mapping*) our model won't expect, so we'll remove them.<br>
https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.remove_columns

In [ ]:
data = tokenized_datasets.remove_columns(["offset_mapping", 
                                          "overflow_to_sample_mapping"])

The last preparation step is to convert the Hugging Face **Dataset** objects into a Tensorflow-compatible datasets.<br>
https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.to_tf_dataset<br>
https://huggingface.co/docs/datasets/main/en/use_with_tensorflow#when-to-use-totfdataset

In [ ]:
train_set = data['train'].to_tf_dataset(batch_size=batch_size)
validation_set = data['validation'].to_tf_dataset(batch_size=batch_size)

We can now download a pre-trained model for fine-tuning. Just like we did with the tokenizer, we'll use an Auto Class to download the right model. In this case, we're using **TFAutoModelForQuestionAnswering**. This will download a Tensorflow implementation of the pre-trained model with a question answering head on it.<br>

The head in this case is a dense layer that returns *start_logits* and *end_logits*. We can take the argmax of each to determine the start and end of the answer span (see model code for details).<br>
https://huggingface.co/docs/transformers/main/en/model_doc/auto#transformers.TFAutoModelForQuestionAnswering<br>
https://github.com/huggingface/transformers/blob/main/src/transformers/models/roberta/modeling_tf_roberta.py#L1629

In [ ]:
model = TFAutoModelForQuestionAnswering.from_pretrained(model_name)

The following method attempts to answer a question given a context. It tokenizes the question and context, runs it through the model, takes the argmax of the start and end logits, and uses the result to extract an answer span from the context.

In [ ]:
def get_answer(tokenizer, model, question, context):
  inputs = tokenizer([question], [context], return_tensors="np")
  outputs = model(inputs)
  start_position = tf.argmax(outputs.start_logits, axis=1)
  end_position = tf.argmax(outputs.end_logits, axis=1)
  answer = inputs["input_ids"][0, int(start_position) : int(end_position) + 1]
  return tokenizer.decode(answer).strip()

While the model body (the pre-trained *distilroberta-base* model) is trained, the head is not. So if we try to use our model to answer a question, it should fail or perform poorly (your output will differ because of different initial head weight values).

In [ ]:
c = "Sarah went to The Mirthless Cafe last night to meet her friend."
q = "Where did Sarah go?"
get_answer(tokenizer, model, q, c)

In [ ]:
# https://www.tensorflow.org/guide/mixed_precision
keras.mixed_precision.set_global_policy("mixed_float16")

# Use a learning rate recommended by the BERT authors.
# https://github.com/google-research/bert
model.compile(optimizer=keras.optimizers.Adam(learning_rate=3e-5))

We'll now fine-tune the model. Note that we didn't freeze the layers of the pre-trained body, so its weights will be tuned along with the head's weights.<br>

Because the body is already pre-trained, we don't need a lot of epochs. 2-4 is typically enough (BERT authors recommend 4). Here, we're using 1 to demonstrate the power of pre-training.<br>

**Note:** If you have GPU enabled and you're using Colab's free tier, the training time can be all over the place depending on which GPU you get assigned (anywhere from 20 minutes to an hour).

In [ ]:
model.fit(train_set, validation_data=validation_set, epochs=1)

After completing our fine-tuning, we should now have a decent extractive question answering model.

In [ ]:
c = "Sarah went to The Mirthless Cafe last night to meet her friend."
q = "Where did Sarah go?"
get_answer(tokenizer, model, q, c)

In [ ]:
q = "Who did Sarah meet?"
get_answer(tokenizer, model, q, c)

In [ ]:
q = "When did Sarah meet her friend?"
get_answer(tokenizer, model, q, c)

In [ ]:
q = "Who went to the restaurant?"
get_answer(tokenizer, model, q, c)

But as we saw earlier, extractive question answering has its limits.

In [ ]:
# Asking a logic teaser question is difficult despite the
# answer being available. To be fair, there is ambiguity here.
q = "Who did Sarah's friend meet?"
get_answer(tokenizer, model, q, c)

In [ ]:
# The model can't determine when a question can't be 
# answered. Some question answering datasets explicitly 
# train for this.
q = "How did Sarah get to the restaurant?"
get_answer(tokenizer, model, q, c)

In [ ]:
# The model isn't generative, either.
q = "What is a possible reason for why Sarah met her friend?"
get_answer(tokenizer, model, q, c)

But despite this model's limitations, I hope this shows the power of pre-training and how fast you can get something cool and useful up and running.<br>

I encourage you to make an account on Hugging Face and push your model to the hub. Learn how here:
https://huggingface.co/docs/transformers/model_sharing


We're just scratching the surface of question answering. Indeed you could dedicate an entire career to it. Areas to explore:
- Right now, we have to supply the context along with the question. A more sophisticated system would load all relevant documents into some database and search over it for an appropriate context/passage and then extract the answer from it. If multiple answers are extracted, then maybe some ranking system can be included as well.
- Another enhancement is extracting answers from different kinds of data. Beyond text, there are images, audio, graphs, tables, charts, etc.
- Abstractive answering involves composing answers (possibly multiple lines) rather than extracting them. Open book means having the system search for the answer first in a database, then composing an answer based on what it's found. Closed book means the model relies on its internal knowledge only. This is what a large language model like GPT-3 would do.
<br>

Speaking of GPT-3, let's play with a few more prompts. At this point, we'll switch to the [OpenAI Playground](https://beta.openai.com/playground) and try out the prompts below. Check out the module video for commentary.<br>

Before you can try out these prompts yourself, you'll need to open an account an [OpenAI](https://openai.com/api/). At the time of this recording, OpenAI was providing a few dollars of credit to get started. It's more than enough to run the prompts below.

## GPT-3 prompts to try out:

---
He said hello<br>
She said bonjour<br>
He said goodbye<br>
She said<br>

---
My name is Kilgore. I run a revenge-for-hire business called Delightful Reprisals. I billed my customer the wrong amount. Write an apology email to the customer and sign it with my name and company. Include a reason blaming Elon Musk.

---
He brought three pies to the office. He gave one to his co-workers, and threw one at his boss' face. How many pies did he have left?

---
Answer in the style of Jeopardy. He was the 26th President of the United States.

---
Sarah went to The Mirthless Cafe last night to meet her friend.<br>

Print each answer to the following questions on separate lines.<br>

Where did Sarah go?<br>
Who did Sarah meet?<br>
When did Sarah meet her friend?<br>
How did Sarah get to the restaurant?<br>
What's a possible reason why they met?<br>

---
What did Marie Antoinette say about avocado toast?

---
What's a word that rhymes with money?

---
What's a word that rhymes with Kafkaesque?

---
Write a haiku about The Terminator.

---
Write a limerick about The Terminator.

---

There are open-source generative decoder models available on Hugging Face as well. Among them are:
- [GPT-2](https://huggingface.co/gpt2)
- [EleutherAI/gpt-neo-1.3B](https://huggingface.co/EleutherAI/gpt-neo-1.3B)
- [bigscience/bloom-560m](https://huggingface.co/bigscience/bloom-560m)


**Be on the lookout for GPT-4!**

# Further Exploration

OpenAI API docs to learn how to build products using their models:<br>
https://openai.com/api/<br>
https://beta.openai.com/docs/introduction

A catalog of transformer models:<br>
https://amatriain.net/blog/transformer-models-an-introduction-and-catalog-2d1e9039f376/


Wordpiece and Sentencepiece:<br>
https://huggingface.co/course/chapter6/6?fw=pt<br>
https://github.com/google/sentencepiece


**Papers**<br>
Attention Is All You Need (original Transformer paper): https://arxiv.org/abs/1706.03762<br>

The Annotated Transformer: http://nlp.seas.harvard.edu/annotated-transformer/<br>

GPT-3: https://arxiv.org/abs/2005.14165<br>

BERT: https://arxiv.org/abs/1810.04805<br>

RoBERTa paper: https://arxiv.org/abs/1907.11692<br>

ALBERT paper: https://arxiv.org/abs/1909.11942<br>

DistilBert paper: https://arxiv.org/abs/1910.01108<br>

Electra paper: https://arxiv.org/abs/2003.10555<br>

XLM: https://arxiv.org/abs/1901.07291<br>